<font size="6"><b><i>Brian2</i> Simulation of Calcium Based Plasticity in a Spiking Model</font>

                                                                           Hanyu Li
                                                                           03/25/2016

In [1]:
import sys
sys.path.append("../")
from IPython.display import display, Math, Latex

# Background and Previous Work

## 1. Calcium Plasticity in Spiking Model

synaptic efficacy changes induced by calcium


<font size="4"><b>Calcium Based Plasticity Rule Described by Grauper-Brunel Model(Grauper et.al, 2012):</b> 

\begin{equation}
\tau \frac{d\rho}{dt} = -\rho (1-\rho)(\rho_*- \rho)
        + \gamma_p (1-\rho)\Theta[c(t)-\theta_p]
        - \gamma_d \rho \Theta[c(t)-\theta_d] + Noise(t)
\end{equation}



<div style="align: left; text-align:center;">
<img src="pictures/calcium_dynamics1.png">
<div class="caption">Fig.1 Order of Pre-Post Spike Pair causes potentiation or depression 
</div>

<br>
<br>

<b>Variation in Higgins(2014)</b>

\begin{equation}
\tau \frac{d\rho}{dt} = -\frac{\partial U(\rho)} {\partial \rho}
        - \gamma_d \rho \Theta[c(t)-\theta_d] 
        + \gamma_p (1-\rho)\Theta[c(t)-\theta_p]
        + \sigma \sqrt{\tau} \sqrt{\Theta(t)[c(t) - \theta_D] + \Theta[c(t)-\theta_P]} \eta(t)
\end{equation}

in which

$U(\rho) = \frac{1}{4} \rho^2 (1-\rho)^2, \ \eta(t)\ is\ Gaussian\ noise$
<div style="align: left; text-align:center;">
<img src="pictures/calcium_dynamics2.png", height=350, width=500>
<div class="caption">Fig.2 Pre-Post Spike Train and Synaptic Efficacy Changes 
</div>

## 2. Rate Model Plasticity and Change in Firing Rate Distribution 

<font size="4"><b>Rate Model and Learning Rule inference in Sukbin et.al, 2016:</b> 

\begin{equation}
\tau_E  \frac{dr_i^E}{dt} = - r_i^E + 
    \Phi_E ( \sum_{j=1}^{N_E} W_{ij}^{EE} r_j^E - \sum_{j=1}^{N_I} W_{ij}^{EI} r_j^I + I_i^{EX})
\end{equation}

\begin{equation}
\tau_I  \frac{dr_i^I}{dt} = - r_i^I + \Phi_I ( \sum_{j=1}^{N_E} W_{ij}^{IE} r_j^E + I_i^{EX})
\end{equation}

in which $\Phi$ is the f-I transfer function

Plasticity obeys 

Plasticity is described by:

\begin{equation}
\Delta W_{ij}^{EE} \leftarrow \Delta W_{ij}^{EE} - \frac{1}{N_E} \sum_{j=1}{N_E}\Delta W_{ij}^{EE}
\end{equation}

Familiar inputs 
<div style="align: left; text-align:center;">
<img src="pictures/sukbin2.png", height=350, width=500>
<div class="caption">Fig.3 Comparison between simulated and experimental data 
</div>


<br>
<br>
Familiar inputs 
<div style="align: left; text-align:center;">
<img src="pictures/sukbin1.png", height=350, width=500>
<div class="caption">Fig.3 Comparison between simulated and experimental data 
</div>


# Introduction

Aim: Show the effect Calcium Plasticity in a spiking neural network to reproduce the effect of learning on 

In [2]:
from d3js_network import show_network
show_network(template='connectivity_E_200_I_50.template',
            connectivity='connectivity_E_200_I_50.json',  w=900,h=900)
#show_network(template='connectivity_E_500_I_125.template',
#            connectivity='connectivity_E_500_I_125.json',  w=900,h=900)

In [ ]:
from core import *

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
%cd ~/pypath/spike_model/
%matplotlib inline
%run core.py


$$c = \sqrt{a^2 + b^2}$$